In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/BUP_Video_Classification/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/BUP_Video_Classification


In [3]:
# ==============================================================================
# BUŇKA 1: INICIALIZACE PROSTŘEDÍ (LOKÁLNÍ VERZE)
# (Spusťte vždy jako první po spuštění notebooku)
# ==============================================================================
import os
import sys
from dotenv import load_dotenv

# 1. Nastavení hlavních cest
BASE_PROJECT_PATH = os.getcwd()
print(f"Pracovní složka projektu: {BASE_PROJECT_PATH}")

# 2. Nastavení odvozených cest
SRC_PATH = os.path.join(BASE_PROJECT_PATH, "src")
DATA_PATH = os.path.join(BASE_PROJECT_PATH, "data")
MODEL_INPUTS_PATH = os.path.join(DATA_PATH, "model_inputs")
MODELS_PATH = os.path.join(BASE_PROJECT_PATH, "models") # Nová složka pro .pkl modely
RESULTS_PATH = os.path.join(BASE_PROJECT_PATH, "results") # Nová složka pro .csv s výsledky

# Vytvoříme složky, pokud neexistují
#os.makedirs(MODEL_INPUTS_PATH, exist_ok=True)
#os.makedirs(MODELS_PATH, exist_ok=True)
#os.makedirs(RESULTS_PATH, exist_ok=True)


# 3. Přidání složky 'src' do sys.path (aby fungovaly importy)
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"Složka 'src' přidána do sys.path")
else:
    print(f"Složka 'src' je již v sys.path")

# 4. Načtení API klíče z .env souboru
load_dotenv()
API_KEY_CHECK = os.getenv('OPENAI_API_KEY')
if API_KEY_CHECK and len(API_KEY_CHECK) > 10:
    print("OPENAI_API_KEY úspěšně načten z .env souboru.")
else:
    print("VAROVÁNÍ: OPENAI_API_KEY nenalezen.")

print("\n--- Prostředí je připraveno ---")

Pracovní složka projektu: /content/drive/MyDrive/BUP_Video_Classification
Složka 'src' přidána do sys.path
OPENAI_API_KEY úspěšně načten z .env souboru.

--- Prostředí je připraveno ---


In [7]:
# ==============================================================================
# BUŇKA 2: GLOBÁLNÍ IMPORTY A NASTAVENÍ
# ==============================================================================
import pandas as pd
import numpy as np
import importlib
import os
import pickle
import openai

# Import našich vlastních modulů
import src.data_preparation as dp
import src.video_processing as vp
import src.llm_generation as lg
import src.feature_engineering as fe
import src.modeling as md
import src.plotting as pl
import src.xai as xai

# Nastavení pro hezčí výpisy
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 150)

print("Všechny moduly naimportovány.")

Všechny moduly naimportovány.


In [ ]:
# ==============================================================================
# BUŇKA 3: FÁZE 1 - PŘÍPRAVA DAT
# (Tuto buňku stačí spustit jen jednou)
# ==============================================================================
print("--- FÁZE 1: PŘÍPRAVA DAT ---")
importlib.reload(dp)

selected_classes_list = [
    'ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'Basketball',
    'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BrushingTeeth',
    'CleanAndJerk', 'CliffDiving', 'CuttingInKitchen', 'Fencing', 'GolfSwing'
]

train_split_file = os.path.join(DATA_PATH, "train_split.csv")
if os.path.exists(train_split_file):
    print("Soubory 'train_split.csv' a 'test_split.csv' již existují. Přeskakuji Fázi 1.")
else:
    print("Vytvářím mini-dataset...")
    dp.create_mini_dataset(
        base_data_path=DATA_PATH,
        selected_classes=selected_classes_list,
        full_dataset_folder="UCF101_full/UCF-101" # Ujistěte se, že tato cesta je správná
    )

    print("Vytvářím train/test splity...")
    dp.create_group_splits(base_data_path=DATA_PATH)
    print("Fáze 1 dokončena.")

In [ ]:
# ==============================================================================
# BUŇKA 4: FÁZE 2 - GENEROVÁNÍ POPISŮ (LLM)
# ==============================================================================
print("--- FÁZE 2: GENEROVÁNÍ POPISŮ (LLM) ---")
importlib.reload(lg)
importlib.reload(vp)

lg.generate_all_descriptions(
    data_path=DATA_PATH,
    num_frames_per_video=3, # Začněte s 3 snímky
    rate_limit_pause=0.5
)

print("\n--- Fáze 2 je kompletní ---")

In [ ]:
# ==============================================================================
# BUŇKA 5: FÁZE 3 - FEATURE ENGINEERING (VNOŘENÍ A AGREGACE)
# ==============================================================================
print("--- FÁZE 3: FEATURE ENGINEERING ---")
importlib.reload(fe)

fe.process_and_embed_all(data_path=DATA_PATH)

print("\n--- Fáze 3 je kompletní ---")

In [ ]:
# ==============================================================================
# BUŇKA 6: FÁZE 4 - KLASIFIKACE (TRÉNOVÁNÍ MODELŮ )
# ==============================================================================
print("--- FÁZE 4: TRÉNOVÁNÍ KLASIFIKÁTORŮ ---")

# Znovu načteme modul, aby se projevily změny
importlib.reload(md)

# Cesty pro výstupy (proměnné jsou z Buňky 1)
models_output_path = os.path.join(MODELS_PATH, "openai")
results_output_path = os.path.join(RESULTS_PATH, "final_modeling_results_openai.csv")
model_inputs_path_openai = os.path.join(DATA_PATH, "model_inputs") # Složka z Fáze 3

os.makedirs(models_output_path, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

# 1. Načtení všech dat
model_data = md.load_data_for_modeling(
    model_inputs_path=model_inputs_path_openai,
    data_path=DATA_PATH
)

# 2. Spuštění celé pipeline (pokud model_data byla načtena)
if model_data:
    results_df = md.run_modeling_pipeline(
        data=model_data,
        models_output_path=models_output_path,
        results_output_path=results_output_path
    )

    # 3. Zobrazení výsledků
    print("\n--- FINÁLNÍ VÝSLEDKY MODELOVÁNÍ ---")
    results_df_sorted = results_df.sort_values(by="F1 (Weighted)", ascending=False)
    display(results_df_sorted)
else:
    print("CHYBA: Data pro modelování nebyla načtena, trénování se nespustí.")

In [ ]:
# ==============================================================================
# BUŇKA 7: FÁZE 6 - VIZUALIZACE VÝSLEDKŮ
# ==============================================================================
print("--- FÁZE 6: VIZUALIZACE VÝSLEDKŮ ---")

# Znovu načteme modul
importlib.reload(pl)

# results_df máme v paměti z předchozí buňky
if 'results_df' in locals():
    pl.plot_results_heatmap(results_df)
else:
    # Pokud jste restartovali, načtěte výsledky z CSV
    results_path = os.path.join(RESULTS_PATH, "final_modeling_results_openai.csv")
    if os.path.exists(results_path):
        results_df = pd.read_csv(results_path)
        pl.plot_results_heatmap(results_df)
    else:
        print("Nebyly nalezeny žádné výsledky k vizualizaci. Spusťte prosím Buňku 6.")

In [ ]:
# ==============================================================================
# BUŇKA 9: FÁZE 5 - VÝPOČET AOPC (VELMI POMALÉ)
# ==============================================================================

# Zkontrolujeme, jestli máme všechna data z předchozích buněk
if 'model' in locals() and 'model_data' in locals() and 'tfidf_phrase_data' in locals():
    print("--- FÁZE 5: VÝPOČET AOPC ---")

    N_VIDEOS_FOR_AOPC = 5 # Začněte s 5. Plná analýza by měla mít 20+

    # 1. Získáme seznam testovacích videí, labelů a cest
    test_mask = (np.load(os.path.join(MODEL_INPUTS_PATH, "splits.npy")) == 'test')
    all_relative_paths = np.load(os.path.join(MODEL_INPUTS_PATH, "relative_paths.npy"), allow_pickle=True)

    test_video_paths = [os.path.join(DATA_PATH, p) for p in all_relative_paths[test_mask][:N_VIDEOS_FOR_AOPC]]
    test_labels = model_data['y_test'][:N_VIDEOS_FOR_AOPC]

    # 2. Znovu získáme surová data pro VŠECHNA testovací videa
    print(f"Generuji surová XAI data pro {N_VIDEOS_FOR_AOPC} videí...")
    xai_data_list = []
    for video_path in tqdm(test_video_paths, desc="Generování XAI dat"):
        xai_data_list.append(
            xai.get_raw_data_for_video(
                video_path=video_path,
                llm_client=llm_client,
                embed_client=embed_client,
                num_frames=3
            )
        )

    # 3. Zjistíme režim nejlepšího modelu
    if "phrase" in best_model_row['Feature Set']:
        mode = 'phrase'
        tfidf_data = tfidf_phrase_data
        aopc_title = f"AOPC pro {best_model_name} (Phrase)"
    else:
        mode = 'wordsplit'
        tfidf_data = tfidf_wordsplit_data
        aopc_title = f"AOPC pro {best_model_name} (WordSplit)"

    # 4. Spustíme výpočet AOPC
    print(f"Počítám AOPC pro {N_VIDEOS_FOR_AOPC} videí...")
    aopc_results_df = xai.calculate_aopc(
        model=model,
        video_list=test_video_paths,
        label_list=test_labels,
        xai_data_list=xai_data_list,
        tfidf_data=tfidf_data,
        mode=mode,
        feature_set=best_model_row['Feature Set'],
        k_max=10 # Analyzujeme odstranění až 10 fragmentů
    )

    # 5. Vykreslíme AOPC křivku
    print("\n--- VÝSLEDEK AOPC ---")
    display(aopc_results_df.head())
    importlib.reload(pl)
    pl.plot_aopc_curve(aopc_results_df, title=aopc_title)

else:
    print("CHYBA: Chybí potřebné proměnné z předchozích buněk (model, model_data, tfidf_data).")
    print("Spusťte prosím buňky 6, 7 a 8.")